# คำเเนะนำ

*   ควรจะโหลดข้อมูลให้ครบ
*   เชื่อม google drive




**สิ่งที่จะใช้**

*    Embedding model คือ multilingual-e5-large-instruct
*    vector database  ของ Chroma db
*    RAG ของ langchain


ทำการเชื่อม google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


โหลดสิ่งที่จำเป็น

In [ ]:
!pip install transformers torch chromadb
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q langchain transformers chromadb requests
!pip install -q langchain chromadb requests

In [ ]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q langchain transformers chromadb requests
!pip install -q langchain chromadb requests


ใช้โมเดล huggingface/multilingual-e5-large-instruct  ในการ Embed

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import json

# โหลดโมเดลและ Tokenizer
model_name = "intfloat/multilingual-e5-large-instruct"  # ชื่อโมเดลจาก Hugging Face
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# ฟังก์ชันที่ใช้ในการ embedding ข้อมูล
def embed_recommendations(categorized_recommendations):
    for rec in categorized_recommendations:
        text = rec['text']
        # แปลงข้อความเป็น token
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=512)
        # ใช้โมเดลในการคำนวณ embedding
        with torch.no_grad():
            outputs = model(**inputs)
        # คำนวณค่า embedding จาก hidden state
        embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()
        rec['embedding'] = embedding.tolist()  # แปลง numpy array เป็น list
        print(f"✅ ข้อความ: '{text}' ได้รับ embedding แล้ว")

    return categorized_recommendations

# ฟังก์ชันที่ใช้ในการบันทึกข้อมูลลงในไฟล์ JSON
def save_embeddings_to_file(data, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print(f"✅ บันทึกไฟล์ที่มีข้อมูล embedding ลงใน: {output_file_path}")

# ฟังก์ชันที่ใช้ในการอ่านข้อมูลจากไฟล์ JSON
def read_recommendations(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    return data

# 🚀 Main
file_path = '/content/drive/MyDrive/LockLearn/recommendations_with_categories.json'  # ระบุ path ของไฟล์ที่จัดหมวดหมู่แล้ว
recommendations = read_recommendations(file_path)

# ทำการ embedding ข้อมูลคำแนะนำ
recommendations_with_embeddings = embed_recommendations(recommendations)

# บันทึกข้อมูลที่ embedding ลงในไฟล์แยก
embedding_output_file = '/content/drive/MyDrive/LockLearn/recommendations_with_embeddings.json'
save_embeddings_to_file(recommendations_with_embeddings, embedding_output_file)

print("✅ เสร็จสิ้นการทำ embedding และบันทึกข้อมูลลงไฟล์แยก")


สร้าง vector database

In [ ]:
import chromadb
import json

# 🚀 โหลดไฟล์ embeddings ที่มีอยู่แล้ว
embedding_file = '/content/drive/MyDrive/LockLearn/recommendations_with_embeddings.json'
with open(embedding_file, 'r', encoding='utf-8') as f:
    recommendations_with_embeddings = json.load(f)

# 🚀 สร้าง client และ collection
db_path = '/content/drive/MyDrive/LockLearn/chromadb_database'  # ระบุ path ของฐานข้อมูล
client = chromadb.PersistentClient(path=db_path)

# ลบ Collection เก่า (ถ้ามี)
if "recommendations" in [col.name for col in client.list_collections()]:
    client.delete_collection(name="recommendations")

# สร้าง Collection ใหม่
collection = client.get_or_create_collection(name="recommendations")

# 🚀 เพิ่มข้อมูลที่มี embedding ลงใน Chroma DB
for i, rec in enumerate(recommendations_with_embeddings):
    collection.add(
        documents=[rec['text']],
        embeddings=[rec['embedding']],
        metadatas=[{"category": rec.get('category', 'Unknown')}],
        ids=[f"rec_{i}"]
    )

print("✅ เก็บคำแนะนำทั้งหมดที่มี embedding ลง Chroma DB ใน Google Drive เรียบร้อย")
